In [105]:
import plotly.express as px
import pandas as pd
import plotly.io as pio
import imageio
from tqdm import tqdm
import json
from datetime import datetime
import pytz

In [106]:
# load parquet file to dataframe
df = pd.read_parquet('../data/log-interpol/interpol.parquet')

In [107]:
df.head()

team          56971933-150d-11ef-9427-2bf640545472             \
dim                                            lat        lon   
1716037344642                            49.018445  12.096630   
1716037345642                            49.018444  12.096632   
1716037346642                            49.018442  12.096636   
1716037347642                            49.018440  12.096640   
1716037348642                            49.018438  12.096644   

team          a4eebb09-1514-11ef-b019-73f1875d90c5             \
dim                                            lat        lon   
1716037344642                            49.018377  12.096603   
1716037345642                            49.018377  12.096603   
1716037346642                            49.018377  12.096603   
1716037347642                            49.018377  12.096603   
1716037348642                            49.018377  12.096603   

team          284f0e6b-1514-11ef-a3d1-710a0d2c97dd             \
dim                                            lat        lon   
1716037344642                            49.018446  12.096713   
1716037345642                            49.018445  12.096713   
1716037346642                            49.018445  12.096713   
1716037347642                            49.018445  12.096713   
1716037348642                            49.018445  12.096714   

team          b1d91f00-1514-11ef-90d9-870e7bed4f26             \
dim                                            lat        lon   
1716037344642                            49.018386  12.096590   
1716037345642                            49.018388  12.096587   
1716037346642                            49.018388  12.096585   
1716037347642                            49.018388  12.096585   
1716037348642                            49.018388  12.096584   

team          50b848eb-150d-11ef-a477-15bc960d382e             
dim                                            lat        lon  
1716037344642                            49.018418  12.096596  
1716037345642                            49.018418  12.096595  
1716037346642                            49.018418  12.096595  
1716037347642                            49.018418  12.096595  
1716037348642                            49.018418  12.096595

In [108]:
# get list of teams
teams = list(set(c[0] for c in df.columns))
teams

['50b848eb-150d-11ef-a477-15bc960d382e',
 '56971933-150d-11ef-9427-2bf640545472',
 'a4eebb09-1514-11ef-b019-73f1875d90c5',
 'b1d91f00-1514-11ef-90d9-870e7bed4f26',
 '284f0e6b-1514-11ef-a3d1-710a0d2c97dd']

In [109]:
# get catch data
data_caught = json.load(open('../data/log-export/team_caught.json'))

In [110]:
# dictionary of caught timestamps
caught_timestamps = {}
for entry in data_caught:
    entry['timestamp'] = entry['timestamp'][:-4]
    entry['timestamp'] = entry['timestamp']
    dt = datetime.strptime(entry['timestamp'], "%Y-%m-%d %H:%M:%S").replace(tzinfo=pytz.UTC)
    caught_timestamps[entry['runaway_active_user']] = int(dt.timestamp() * 1000)
# initial hunting team
entry = data_caught[-1]
caught_timestamps

{'56971933-150d-11ef-9427-2bf640545472': 1716041758000,
 'b1d91f00-1514-11ef-90d9-870e7bed4f26': 1716041166000,
 'a4eebb09-1514-11ef-b019-73f1875d90c5': 1716039541000}

In [111]:
# get initial hunter
inital_hunter = entry['hunter_active_user']
caught_timestamps[inital_hunter] = 0

In [112]:
# Render the frames figures
figs = []
for t in tqdm(df.index[::10]):
    color = []
    for team in teams:
        if team == inital_hunter:
            color.append('hunter')
        elif team in caught_timestamps.keys():
            if caught_timestamps[team] < t:
                color.append('hunter')
            else:
                color.append(team)
        else:
            color.append(team)
    data = {'lat': [df[team, 'lat'][t] for team in teams],
            'lon': [df[team, 'lon'][t] for team in teams]}
    df_plot = pd.DataFrame(data)

    # Create plot
    fig = px.scatter_mapbox(df_plot, lat="lat", lon="lon", color=color, zoom=15)
    fig.update_layout(mapbox_style="carto-positron", mapbox_center_lat=df[teams[0], 'lat'][df.index[0]], mapbox_center_lon=df[teams[0], 'lon'][df.index[0]], showlegend=False)
    figs.append(fig)

100%|██████████| 447/447 [00:23<00:00, 18.67it/s]


In [113]:
# Convert figures to images
frames = []
for fig in tqdm(figs):
    img_bytes = pio.to_image(fig, format="png")
    img = imageio.v2.imread(img_bytes)
    frames.append(img)

100%|██████████| 447/447 [07:50<00:00,  1.05s/it]


In [115]:
# repeat last frame for 2 seconds
for i in range(20):
    frames.append(frames[-1])

In [116]:
# Save frames as a video
imageio.mimsave("video.mp4", frames, fps=30)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (700, 500) to (704, 512) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
